In [ ]:
# Setup Environment
import pandas as pd
import seaborn as sns
from scipy import stats
import matplotlib.pyplot as plt

sns.set_style('darkgrid')
sns.set_context('talk')

In [ ]:
# Read-in data
df = pd.read_csv('https://raw.githubusercontent.com/aprescot1977/Thinkful/master/no2_master.csv')
df.rename(columns={'Daily Max 1-hour NO2 Concentration':'Daily Max 1-hr [NO2] in ppb'}, inplace=True)
pollutant_no2 = 'Daily Max 1-hr [NO2] in ppb'

#### Hypothesis 1: Salt Lake City NO2 over 20 year period (1999 - 2019 in 5 year increments)

In [ ]:
# Visualize distributions for SLC
years = list(df['year'].unique())

ii = 0
fig, axes = plt.subplots(1,len(years), figsize=(35, 8))
while ii<len(years):
    ax = sns.distplot(df[(df['year']==years[ii])&(df['site']=='SLC')][pollutant_no2], ax=axes[ii], color = 'darkslategrey')
    ax.axvline(df[(df['year']==years[ii])&(df['site']=='SLC')][pollutant_no2].mean(), color = 'k', linestyle='dashed')
    ax.axvline(df[(df['year']==years[ii])&(df['site']=='SLC')][pollutant_no2].median(), color = 'r', linestyle='dashed')
    ax.set_title('%s (black = mean, red = median)' % str(years[ii]))
    ii +=1

df[df['site']=='SLC'].groupby(['year']).agg('count')

In [ ]:
# Check normality for SLC distributions
[print('{} statistics: {}'.format(ii, stats.describe(df[(df['year']==ii)&(df['site']=='SLC')][pollutant_no2]))) for ii in years];
[print('{} Shapiro-Wilk: {}'.format(ii,  stats.shapiro(df[(df['year']==ii)&(df['site']=='SLC')][pollutant_no2]))) for ii in years];

In [ ]:
# Kruskal-Wallis and One Way ANOVA (Discuss with Mukesh) for SLC
print(stats.kruskal(df[(df['year']==1999)&(df['site']=='SLC')][pollutant_no2],
                    df[(df['year']==2004)&(df['site']=='SLC')][pollutant_no2], 
                    df[(df['year']==2009)&(df['site']=='SLC')][pollutant_no2], 
                    df[(df['year']==2014)&(df['site']=='SLC')][pollutant_no2],
                    df[(df['year']==2018)&(df['site']=='SLC')][pollutant_no2]))

print(stats.f_oneway(df[(df['year']==1999)&(df['site']=='SLC')][pollutant_no2],
                    df[(df['year']==2004)&(df['site']=='SLC')][pollutant_no2], 
                    df[(df['year']==2009)&(df['site']=='SLC')][pollutant_no2], 
                    df[(df['year']==2014)&(df['site']=='SLC')][pollutant_no2],
                    df[(df['year']==2018)&(df['site']=='SLC')][pollutant_no2]))

[print('{} SLC NO2 median: {}'.format(ii, df[(df['year']==ii)&(df['site']=='SLC')][pollutant_no2].median())) for ii in years];

In [ ]:
# Plot NO2 time course for SLC (1999-2019)
fig, axes = plt.subplots(1,1,figsize = (20, 10))
ax = sns.boxplot(y = pollutant_no2, x = 'year', data = df[df['site']=='SLC'])
ax.axhline(100, color = 'k', linestyle='dashed');
ax.set_title('Salt Lake City ' + pollutant_no2 + ' for 6 separate years');
style = dict(size=15, color='k');
ax.text(3, 102, "NAAQS One Hour Maximum Standard Level", **style);

#### Hypothesis 2: Honolulu NO2 over 20 year period (1999 - 2019 in 5 year increments)

In [ ]:
# Visualize distributions for HON
years = list(df['year'].unique())

ii = 0
fig, axes = plt.subplots(1,len(years), figsize=(35, 8))
while ii<len(years):
    ax = sns.distplot(df[(df['year']==years[ii])&(df['site']=='HON')][pollutant_no2], ax=axes[ii], color = 'darkslategrey')
    ax.axvline(df[(df['year']==years[ii])&(df['site']=='HON')][pollutant_no2].mean(), color = 'k', linestyle='dashed')
    ax.axvline(df[(df['year']==years[ii])&(df['site']=='HON')][pollutant_no2].median(), color = 'r', linestyle='dashed')
    ax.set_title('%s (black = mean, red = median)' % str(years[ii]))
    ii +=1

df[df['site']=='HON'].groupby(['year']).agg('count')

In [ ]:
# Check normality for HON distributions
[print('{} statistics: {}'.format(ii, stats.describe(df[(df['year']==ii)&(df['site']=='HON')][pollutant_no2]))) for ii in years];
[print('{} Shapiro-Wilk: {}'.format(ii,  stats.shapiro(df[(df['year']==ii)&(df['site']=='HON')][pollutant_no2]))) for ii in years];

In [ ]:
# Kruskal-Wallis and One Way ANOVA (Discuss with Mukesh) for HON
print(stats.kruskal(df[(df['year']==1999)&(df['site']=='HON')][pollutant_no2],
                    df[(df['year']==2004)&(df['site']=='HON')][pollutant_no2], 
                    df[(df['year']==2009)&(df['site']=='HON')][pollutant_no2], 
                    df[(df['year']==2014)&(df['site']=='HON')][pollutant_no2],
                    df[(df['year']==2018)&(df['site']=='HON')][pollutant_no2]))
print(stats.f_oneway(df[(df['year']==1999)&(df['site']=='HON')][pollutant_no2],
                    df[(df['year']==2004)&(df['site']=='HON')][pollutant_no2], 
                    df[(df['year']==2009)&(df['site']=='HON')][pollutant_no2], 
                    df[(df['year']==2014)&(df['site']=='HON')][pollutant_no2],
                    df[(df['year']==2018)&(df['site']=='HON')][pollutant_no2]))
[print('{} HON NO2 median: {}'.format(ii, df[(df['year']==ii)&(df['site']=='HON')][pollutant_no2].median())) for ii in years];

In [ ]:
# Plot NO2 time course for HON (1999-2019)
fig, axes = plt.subplots(1,1,figsize = (20, 10))
ax = sns.boxplot(y = pollutant_no2, x = 'year', data = df[df['site']=='HON'])
ax.set_title('Honolulu ' + pollutant_no2 + ' for 6 separate years');

#### Hypothesis 3: Salt Lake City versus Honolulu NO2 levels over a 20 year period (1999 to 2019 in 5 year increments)

In [ ]:
# Annual statistics to compare SLC versus HON NO2 levels
[print('{} SLC v HON statistics: {}'.format(ii, stats.kruskal(df[(df['year']==ii)&(df['site']=='SLC')][pollutant_no2], df[(df['year']==ii)&(df['site']=='HON')][pollutant_no2]))) for ii in years];

In [ ]:
# Plot SLC and HON levels for all years
fig, axes = plt.subplots(1,1,figsize = (20, 10))
ax = sns.boxplot(y = pollutant_no2, x = 'site', data = df, hue = 'year')
ax.axhline(100, color = 'k', linestyle='dashed');
ax.set_title('Salt Lake City and Honolulu ' + pollutant_no2 + ' for 6 separate years');
style = dict(size=15, color='k');
ax.text(0.75, 102, "NAAQS One Hour Maximum Standard Level", **style);

#### Hypothesis 4: Salt Lake City NO2 analyzed by season over one year period (Autumn 2018 - Autumn 2019)

In [ ]:
# Seasonal effects: first build a subset of data spanning autumn 2018 to autumn 2019 from master data frame
df_autumn18 = pd.DataFrame(df.loc[df['Date'].between('2018-09-01','2018-11-30')])
df_winter18 = pd.DataFrame(df.loc[df['Date'].between('2018-12-01','2019-02-28')])
df_spring19 = pd.DataFrame(df.loc[df['Date'].between('2019-03-01','2019-05-31')])
df_summer19 = pd.DataFrame(df.loc[df['Date'].between('2019-06-01','2019-08-31')])
df_autumn19 = pd.DataFrame(df.loc[df['Date'].between('2019-09-01','2019-11-30')])

df_autumn18['season'] = 'autumn 2018'
df_winter18['season'] = 'winter 2018'
df_spring19['season'] = 'spring 2019'
df_summer19['season'] = 'summer 2019'
df_autumn19['season'] = 'autumn 2019'

df_season = pd.concat([df_autumn18, df_winter18, df_spring19, df_summer19, df_autumn19])

In [ ]:
# Look at seasonal distribution for SLC
season_list = list(df_season['season'].unique())

ii = 0
fig, axes = plt.subplots(1,len(season_list), figsize=(35, 8))
while ii<len(season_list):
    ax = sns.distplot(df_season[(df_season['season']==season_list[ii])&(df_season['site']=='SLC')][pollutant_no2], ax=axes[ii], color = 'darkslategrey')
    ax.axvline(df_season[(df_season['season']==season_list[ii])&(df_season['site']=='SLC')][pollutant_no2].mean(), color = 'k', linestyle='dashed')
    ax.axvline(df_season[(df_season['season']==season_list[ii])&(df_season['site']=='SLC')][pollutant_no2].median(), color = 'r', linestyle='dashed')
    ax.set_title('%s (black = mean, red = median)' % str(season_list[ii]))
    ii +=1

df_season[df_season['site']=='SLC'].groupby(['season']).agg('count')

In [ ]:
# Check normality for seasonal SLC distributions
[print('{} statistics: {}'.format(ii, stats.describe(df_season[(df_season['season']==ii)&(df_season['site']=='SLC')][pollutant_no2]))) for ii in season_list];
[print('{} shapiro-wilk: {}'.format(ii,  stats.shapiro(df_season[(df_season['season']==ii)&(df_season['site']=='SLC')][pollutant_no2]))) for ii in season_list];

In [ ]:
# Seasonal statistics to compare SLC NO2 level throughout the year
print(stats.kruskal(df_season[(df_season['season']=='autumn 2018')&(df_season['site']=='SLC')][pollutant_no2],
                    df_season[(df_season['season']=='winter 2018')&(df_season['site']=='SLC')][pollutant_no2],
                    df_season[(df_season['season']=='spring 2019')&(df_season['site']=='SLC')][pollutant_no2],
                    df_season[(df_season['season']=='summer 2019')&(df_season['site']=='SLC')][pollutant_no2],
                    df_season[(df_season['season']=='autumn 2019')&(df_season['site']=='SLC')][pollutant_no2]))
print(stats.f_oneway(df_season[(df_season['season']=='autumn 2018')&(df_season['site']=='SLC')][pollutant_no2],
                     df_season[(df_season['season']=='winter 2018')&(df_season['site']=='SLC')][pollutant_no2],
                     df_season[(df_season['season']=='spring 2019')&(df_season['site']=='SLC')][pollutant_no2],
                     df_season[(df_season['season']=='summer 2019')&(df_season['site']=='SLC')][pollutant_no2],
                     df_season[(df_season['season']=='autumn 2019')&(df_season['site']=='SLC')][pollutant_no2]))
[print('{} SLC seasonal NO2 median: {}'.format(ii, df_season[(df_season['season']==ii)&(df_season['site']=='SLC')][pollutant_no2].median())) for ii in season_list];

#### Hypothesis 5: Honolulu NO2 analyzed by season over one year period (Autumn 2018 - Autumn 2019)

In [ ]:
# Look at seasonal distribution for HON
season_list = list(df_season['season'].unique())

ii = 0
fig, axes = plt.subplots(1,len(season_list), figsize=(35, 8))
while ii<len(season_list):
    ax = sns.distplot(df_season[(df_season['season']==season_list[ii])&(df_season['site']=='HON')][pollutant_no2], ax=axes[ii], color = 'darkslategrey')
    ax.axvline(df_season[(df_season['season']==season_list[ii])&(df_season['site']=='HON')][pollutant_no2].mean(), color = 'k', linestyle='dashed')
    ax.axvline(df_season[(df_season['season']==season_list[ii])&(df_season['site']=='HON')][pollutant_no2].median(), color = 'r', linestyle='dashed')
    ax.set_title('%s (black = mean, red = median)' % str(season_list[ii]))
    ii +=1

df_season[df_season['site']=='HON'].groupby(['season']).agg('count')

In [ ]:
# Check normality for seasonal HON distributions
[print('{} statistics: {}'.format(ii, stats.describe(df_season[(df_season['season']==ii)&(df_season['site']=='SLC')][pollutant_no2]))) for ii in season_list];
[print('{} shapiro-wilk: {}'.format(ii,  stats.shapiro(df_season[(df_season['season']==ii)&(df_season['site']=='SLC')][pollutant_no2]))) for ii in season_list];

In [ ]:
# Seasonal statistics to compare HON NO2 level throughout the year
print(stats.kruskal(df_season[(df_season['season']=='autumn 2018')&(df_season['site']=='HON')][pollutant_no2],
                    df_season[(df_season['season']=='winter 2018')&(df_season['site']=='HON')][pollutant_no2],
                    df_season[(df_season['season']=='spring 2019')&(df_season['site']=='HON')][pollutant_no2],
                    df_season[(df_season['season']=='summer 2019')&(df_season['site']=='HON')][pollutant_no2],
                    df_season[(df_season['season']=='autumn 2019')&(df_season['site']=='HON')][pollutant_no2]))
print(stats.f_oneway(df_season[(df_season['season']=='autumn 2018')&(df_season['site']=='HON')][pollutant_no2],
                     df_season[(df_season['season']=='winter 2018')&(df_season['site']=='HON')][pollutant_no2],
                     df_season[(df_season['season']=='spring 2019')&(df_season['site']=='HON')][pollutant_no2],
                     df_season[(df_season['season']=='summer 2019')&(df_season['site']=='HON')][pollutant_no2],
                     df_season[(df_season['season']=='autumn 2019')&(df_season['site']=='HON')][pollutant_no2]))
[print('{} SLC seasonal NO2 median: {}'.format(ii, df_season[(df_season['season']==ii)&(df_season['site']=='HON')][pollutant_no2].median())) for ii in season_list];

In [ ]:
fig, axes = plt.subplots(1,1,figsize = (20, 10))
ax = sns.boxplot(y = pollutant_no2, x = 'site', data = df_season, hue = 'season')
ax.axhline(100, color = 'k', linestyle='dashed');
ax.set_title('Salt Lake City and Honolulu ' + pollutant_no2 + ' Autumn 2018 through Autumn 2019');
style = dict(size=15, color='k');
ax.text(0.75, 102, "NAAQS One Hour Maximum Standard Level", **style);